# Use Case: Climatology - Air temperature over Europe

## Import packages

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from c3s_eqc_automatic_quality_control import download, utils

plt.style.use("seaborn-v0_8-notebook")

## Define parameters

In [ ]:
# Define region
lon_slice = slice(-11, -5)
lat_slice = slice(36, 45)

# Define climatology periods
years_start = [1951, 1961, 1971, 1981, 1991]
years_stop = [1980, 1990, 2000, 2010, 2020]
colors = ["deepskyblue", "green", "gold", "darkorange", "red"]

# Variable name
varname = "tg"

## Define function to cache

In [ ]:
def regionalise_and_dayofyear_reindex(ds, lon_slice, lat_slice):
    # Select region
    ds = utils.regionalise(ds, lon_slice=lon_slice, lat_slice=lat_slice)

    # 15-day rolling mean
    ds_rolled = ds.rolling(time=15, center=True).mean()

    # Extract periods
    datasets = []
    for year_start, year_stop in zip(years_start, years_stop):
        period = f"{year_start}-{year_stop}"
        ds_masked = ds_rolled.where(
            (ds_rolled["time"].dt.year >= year_start)
            & (ds_rolled["time"].dt.year <= year_stop),
            drop=True,
        )
        datasets.append(
            ds_masked.groupby("time.dayofyear").mean().expand_dims(period=[period])
        )
    ds_dayofyear = xr.merge(datasets)

    # Add season (pick any leap year)
    season = xr.DataArray(
        pd.to_datetime(ds_dayofyear["dayofyear"].values - 1, unit="D", origin="2000"),
    ).dt.season
    return ds_dayofyear.assign_coords(season=("dayofyear", season.values))

## Download and cache data

In [ ]:
request = (
    "insitu-gridded-observations-europe",
    {
        "format": "zip",
        "product_type": "ensemble_mean",
        "variable": "mean_temperature",
        "grid_resolution": "0.25deg",
        "period": "full_period",
        "version": "25.0e",
    },
)
ds = download.download_and_transform(
    *request,
    transform_func=regionalise_and_dayofyear_reindex,
    transform_func_kwargs={"lon_slice": lon_slice, "lat_slice": lat_slice},
)

## Define plotting functions

In [ ]:
def plot_maps(da, **kwargs):
    facet = da.plot.pcolormesh(subplot_kws={"projection": ccrs.PlateCarree()}, **kwargs)
    for ax in facet.axs.flatten():
        ax.set_extent(
            [lon_slice.start, lon_slice.stop, lat_slice.start, lat_slice.stop],
            crs=ccrs.PlateCarree(),
        )
        ax.coastlines(lw=1)
        gl = ax.gridlines(draw_labels=True)
        gl.top_labels = gl.right_labels = False
    return facet


def plot_pdf(da, colors=None, **kwargs):
    lines = []
    for i, (period, da) in enumerate(da.groupby("period")):
        hist, bin_edges = np.histogram(
            da, bins=np.linspace(da.min(), da.max(), 50), density=True
        )
        da_hist = xr.DataArray(
            hist, coords={"bins": (bin_edges[1:] + bin_edges[:-1]) / 2}
        )
        da_hist["bins"].attrs = da.attrs
        da_hist.attrs["long_name"] = "Probability Density"
        if colors is not None:
            kwargs["color"] = colors[i]
        lines.append(da_hist.plot.line(x="bins", label=period, **kwargs))
    return lines

## Plot yearly climatology

In [ ]:
facet = plot_maps(
    ds[varname].mean("dayofyear"), col="period", cmap="RdBu_r", robust=True
)
facet.fig.suptitle("Yearly mean", y=1, va="bottom")

## Plot overall PDF

In [ ]:
fig, ax = plt.subplots(1, 1)
lines = plot_pdf(ds[varname], colors=colors, ax=ax, add_legend=False)
plt.legend()

## Plot seasonal climatology and PDF

In [ ]:
fig, axes = plt.subplots(2, 1, sharey=True)
for season, ax in zip(["DJF", "JJA"], axes):
    facet = plot_maps(
        ds[varname].where(ds["season"] == season).mean("dayofyear"),
        col="period",
        cmap="RdBu_r",
        robust=True,
    )
    facet.fig.suptitle(season)

    plot_pdf(
        ds[varname].where(ds["season"] == season),
        colors=colors,
        ax=ax,
        add_legend=False,
    )
    if ax in axes[:-1]:
        ax.set_xlabel("")
    ax.set_title(season)
    ax.legend()